In [3]:
import os
import pandas as pd
from langchain_community.llms import Ollama

C:\Users\cheefoo3\Anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\cheefoo3\Anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [4]:
llm = Ollama(model="llama2")
llm.invoke("The first man on the moon was ...")

'\nThe first man to walk on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon\'s surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That\'s one small step for man, one giant leap for mankind," as he became the first person to set foot on the lunar surface.'

# Read transaction data

In [5]:
file = "./transactions_2022_2023.csv"
df = pd.read_csv(file)

In [6]:
df.columns

Index(['Date', 'Name / Description', 'Expense/Income', 'Amount (EUR)'], dtype='object')

In [18]:
df

,Date,Name / Description,Expense/Income,Amount (EUR)
0,2023-12-30,Belastingdienst,Expense,9.96
1,2023-12-30,Tesco Breda,Expense,17.53
2,2023-12-30,Monthly Appartment Rent,Expense,451.00
3,2023-12-30,Vishandel Sier Amsterdam,Expense,12.46
4,2023-12-29,Selling Paintings,Income,13.63
5,2023-12-29,Spotify Ab By Adyen,Expense,12.19
6,2023-12-23,Tk Maxx Amsterdam Da,Expense,27.08
7,2023-12-22,Consulting,Income,541.57
8,2023-12-22,Aidsfonds,Expense,10.70
9,2023-12-20,Consulting,Income,2641.93


In [7]:
# Get unique transactions in the Name/Description column
unique_transactions = df['Name / Description']
len(unique_transactions)

28

In [17]:
unique_transactions

0                 Belastingdienst
1                     Tesco Breda
2         Monthly Appartment Rent
3        Vishandel Sier Amsterdam
4               Selling Paintings
5             Spotify Ab By Adyen
6            Tk Maxx Amsterdam Da
7                      Consulting
8                       Aidsfonds
9                      Consulting
10        Tls Bv Inz Ov-Chipkaart
11                 Etos Amsterdam
12                    Tesco Breda
13    Beta Boulders Ams Amsterdam
14                         Salary
15       Bouldermuur Bv Amsterdam
16    Birtat Restaurant Amsterdam
17                    Tesco Breda
18                    Freelancing
19                         Tikkie
20                       Blogging
21                   Taxi Utrecht
22                    Tesco Breda
23                 Apple Services
24                    Tesco Breda
25                     Amazon Lux
26             Classpass* Monthly
27     Audible Uk AdblCo/Pymt Gbr
Name: Name / Description, dtype: object

In [12]:
prompt = f"""
Can you add an appropriate category to the following expenses. 
For example: Spotify AB by Adyen - Entertainment, Beta Boulders Ams Amsterdam Nld - Sport, etc.. 
Categories should be less than 4 words. {unique_transactions}
"""

response = llm.invoke(prompt)
response = response.split('\n')

In [13]:
response

['Sure! Here are the categories for each expense in the list:',
 '',
 '1. Belastingdienst - Taxes',
 '2. Tesco Breda - Groceries',
 '3. Monthly Appartment Rent - Housing',
 '4. Vishandel Sier Amsterdam - Shopping',
 '5. Spotify Ab By Adyen - Entertainment',
 '6. Tk Maxx Amsterdam Da - Clothing/Shopping',
 '7. Consulting - Professional Services',
 '8. Aidsfonds - Charity Donations',
 '9. Consulting - Professional Services',
 '10. Tls Bv Inz Ov-Chipkaart - Transportation',
 '11. Etos Amsterdam - Groceries',
 '12. Tesco Breda - Groceries',
 '13. Beta Boulders Ams Amsterdam - Sports/Fitness',
 '14. Salary - Personal Income',
 '15. Bouldermuur Bv Amsterdam - Home Improvements',
 '16. Birtat Restaurant Amsterdam - Dining Out',
 '17. Tesco Breda - Groceries',
 '18. Freelancing - Professional Services',
 '19. Tikkie - Financial Transactions',
 '20. Blogging - Personal Expenses',
 '21. Taxi Utrecht - Transportation',
 '22. Tesco Breda - Groceries',
 '23. Apple Services - Technology',
 '24. Tesc

In [16]:
# Put in dataframe
    # We need an index to put into dataframe
    # So use response.split(\n) to split into multiple elements 
categories_df = pd.DataFrame({'Transaction vs category': response})
categories_df[['Transaction', 'Category']] = categories_df['Transaction vs category'].str.split(' - ', expand=True)
categories_df

,Transaction vs category,Transaction,Category
0,Sure! Here are the categories for each expense...,Sure! Here are the categories for each expense...,None
1,,,None
2,1. Belastingdienst - Taxes,1. Belastingdienst,Taxes
3,2. Tesco Breda - Groceries,2. Tesco Breda,Groceries
4,3. Monthly Appartment Rent - Housing,3. Monthly Appartment Rent,Housing
5,4. Vishandel Sier Amsterdam - Shopping,4. Vishandel Sier Amsterdam,Shopping
6,5. Spotify Ab By Adyen - Entertainment,5. Spotify Ab By Adyen,Entertainment
7,6. Tk Maxx Amsterdam Da - Clothing/Shopping,6. Tk Maxx Amsterdam Da,Clothing/Shopping
8,7. Consulting - Professional Services,7. Consulting,Professional Services
9,8. Aidsfonds - Charity Donations,8. Aidsfonds,Charity Donations


In [31]:
# Remove the numbering eg "1. " from Transaction column
import re
categories_df_all = pd.DataFrame()
def remove_numbering(text):
    return re.sub(pattern, '', text)
pattern = r'^\d+\.\s?'
# Modify the 'Transaction' column in place
categories_df['Transaction'] = categories_df['Transaction'].str.replace(pattern, '', regex=True)
categories_df

,Transaction vs category,Transaction,Category
0,Sure! Here are the categories for each expense...,Sure! Here are the categories for each expense...,None
1,,,None
2,1. Belastingdienst - Taxes,Belastingdienst,Taxes
3,2. Tesco Breda - Groceries,Tesco Breda,Groceries
4,3. Monthly Appartment Rent - Housing,Monthly Appartment Rent,Housing
5,4. Vishandel Sier Amsterdam - Shopping,Vishandel Sier Amsterdam,Shopping
6,5. Spotify Ab By Adyen - Entertainment,Spotify Ab By Adyen,Entertainment
7,6. Tk Maxx Amsterdam Da - Clothing/Shopping,Tk Maxx Amsterdam Da,Clothing/Shopping
8,7. Consulting - Professional Services,Consulting,Professional Services
9,8. Aidsfonds - Charity Donations,Aidsfonds,Charity Donations


In [32]:
# Merge the categories_df_all with the transactions_2022_2023.csv dataframe (df)
df = pd.read_csv("transactions_2022_2023.csv")
df.loc[df['Name / Description'].str.contains("Spotify"), 'Name / Description'] = "Spotify Ab By Adyen"
df = pd.merge(df, categories_df, left_on='Name / Description', right_on='Transaction', how='left')
df

,Date,Name / Description,Expense/Income,Amount (EUR),Transaction vs category,Transaction,Category
0,2023-12-30,Belastingdienst,Expense,9.96,1. Belastingdienst - Taxes,Belastingdienst,Taxes
1,2023-12-30,Tesco Breda,Expense,17.53,2. Tesco Breda - Groceries,Tesco Breda,Groceries
2,2023-12-30,Tesco Breda,Expense,17.53,12. Tesco Breda - Groceries,Tesco Breda,Groceries
3,2023-12-30,Tesco Breda,Expense,17.53,17. Tesco Breda - Groceries,Tesco Breda,Groceries
4,2023-12-30,Tesco Breda,Expense,17.53,22. Tesco Breda - Groceries,Tesco Breda,Groceries
5,2023-12-30,Tesco Breda,Expense,17.53,24. Tesco Breda - Groceries,Tesco Breda,Groceries
6,2023-12-30,Monthly Appartment Rent,Expense,451.00,3. Monthly Appartment Rent - Housing,Monthly Appartment Rent,Housing
7,2023-12-30,Vishandel Sier Amsterdam,Expense,12.46,4. Vishandel Sier Amsterdam - Shopping,Vishandel Sier Amsterdam,Shopping
8,2023-12-29,Selling Paintings,Income,13.63,NaN,NaN,NaN
9,2023-12-29,Spotify Ab By Adyen,Expense,12.19,5. Spotify Ab By Adyen - Entertainment,Spotify Ab By Adyen,Entertainment
